# Building Risk Parity into a Stock Portfolio

Using OpenBB and Riskfolio we will find the risk-weighted allocation on a stock portfolio

Risk Parity will help us allocate our portfolio based on target risk level. The goal is to build a protfolio to maximize returns in a volitile market.

In [2]:
from openbb_terminal.sdk import openbb
import riskfolio as rp 
import pandas as pd

Create list of portfolio tickerts along with the date range of desired historical closing prices. 

Using the OpenBB Comparison Analysis module we can pull daily closing prices of our stocks within the date range

In [3]:
start = "2023-01-22"
end = "2023-05-23"
symbols = ['AAPL','MSFT','AMZN','FTNT','BA','RTX','CVX','CAT','CMI','TXN','JPM','GEHC','LOW','AMT','CEG','ULTA','KURA']
tickers = openbb.stocks.ca.hist(symbols,start, end)

NaN values found in: FTNT.  Backfilling data

Taking the historical prices of each stock we now want to calculate the % change from day-to-day to paint a clearer picture of volitility

In [42]:
returns = tickers.pct_change()[1:]
returns.dropna(how="any", axis=1, inplace=True)

# Building Risk Parrity Portfolio

Creating the Portfolio object to calculate the stats based on the % returns. 

The final variable w_rp_c output contains a dataframe with the % weights for a risk parity portfolio

In [43]:
port = rp.Portfolio(returns=returns)

port.assets_stats(method_mu='hist', method_cov='hist', d=0.94)

port.lowerret = 0.0008
w_rp_c = port.rp_optimization(
    model="Classic",
    rm="MV",
    hist=True,
    rf=0,
    b=None
)

Now that we have the risk parity allocations for each stock, lets build in the total value of the portfolio to get a $ and share total based on the allocation %

We have $16,000 invested in the portfolio. The following code will get us the $ and share total needed to equal the allocation %

In [44]:
port_val = 16_000
w_rp_c["invest_amt"] = w_rp_c * port_val

In [45]:
dollar_amt = w_rp_c

dollar_amt['last_price'] = tickers.iloc[-1]
allocations = dollar_amt

Below we've got a dataframe with the allocation % weights along with invest_amt that gives us a $ amount needed to fulfill the allocation %. 

We've also used the invest_amt and divided it by the last price of the stock to get a total number of shares to purchase to fulfill the allocation %

In [52]:
allocations['shares'] = (allocations.invest_amt / allocations.last_price).astype(int)

allocations.index.name = 'Ticker'

allocations.sort_values (by='weights', ascending =False)


,weights,invest_amt,last_price,shares
Ticker,,,,
MSFT,0.099865,1597.845673,315.260010,5
AAPL,0.080199,1283.178321,171.559998,7
ULTA,0.077580,1241.274267,480.609985,2
FTNT,0.075466,1207.450343,68.089996,17
RTX,0.072149,1154.376785,94.080002,12
GEHC,0.071119,1137.906401,77.190002,14
KURA,0.058064,929.031869,13.930000,66
JPM,0.055180,882.887234,136.589996,6
CEG,0.053756,860.095342,84.040001,10


I like to use brokers like Stash that allow you to invest dollar amounts vs whole shares. 

To know how much I should buy/sell of each stock, I want to calculate the $ difference from the risk parity invest_amt to the current value in the portfolio.

Below I build a dictionary list of the current portfolio value of each stock.

In [47]:
stocks = {'Ticker':['AAPL','MSFT','AMZN','FTNT','BA','RTX','CVX','CAT','CMI','TXN','JPM','GEHC','LOW','AMT','CEG','ULTA','KURA'],
          'Current_Value': [1126.0,1263.0,1057.0,1199.0,894.0,894.0,848.0,764.0,785.0,912.0,934.0,1013.0,950.0,794.0,940.0,901.0,989.0]
}

df=pd.DataFrame(stocks)

current_stocks = df.set_index('Ticker')

current_stocks.head(20)

,Current_Value
Ticker,
AAPL,1126.0
MSFT,1263.0
AMZN,1057.0
FTNT,1199.0
BA,894.0
RTX,894.0
CVX,848.0
CAT,764.0
CMI,785.0


Here I merged the dataframes on the Ticker to get the invest_amt and current_value in a single dataframe. 

I then created a new calculated column 'diff' that calculates the difference between the invest_amt and current_value. 

The diff gives me exact amounts of how much each stock I should sell and then how much to reinvest in. 

In [56]:
mapping = pd.merge(allocations,current_stocks,on='Ticker')

mapping.astype('float').dtypes

mapping['diff'] = mapping['invest_amt'] - mapping['Current_Value']

mapping.sort_values (by='weights', ascending =False)

,weights,invest_amt,last_price,shares,Current_Value,diff
Ticker,,,,,,
MSFT,0.099865,1597.845673,315.260010,5,1263.0,334.845673
AAPL,0.080199,1283.178321,171.559998,7,1126.0,157.178321
ULTA,0.077580,1241.274267,480.609985,2,901.0,340.274267
FTNT,0.075466,1207.450343,68.089996,17,1199.0,8.450343
RTX,0.072149,1154.376785,94.080002,12,894.0,260.376785
GEHC,0.071119,1137.906401,77.190002,14,1013.0,124.906401
KURA,0.058064,929.031869,13.930000,66,989.0,-59.968131
JPM,0.055180,882.887234,136.589996,6,934.0,-51.112766
CEG,0.053756,860.095342,84.040001,10,940.0,-79.904658
